In [1]:
from roler.model import ModelPrior
from roler.distributions import *

prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

In [2]:
from roler.simulation import Simulator

simulator = Simulator()
theta = prior.sample()
stats = simulator.simulate(theta)
stats

R[write to console]: Using GitHub PAT from the git credential store.



Installing the 'roleR' R package from GitHub...


R[write to console]: Skipping install of 'roleR' from a github remote, the SHA1 (cc6546a1) has not changed since last install.
  Use `force = TRUE` to force installation



,hill_abund_1,hill_abund_2,hill_abund_3,hill_abund_4,hill_gen_1,hill_gen_2,hill_gen_3,hill_gen_4,hill_trait_1,hill_trait_2,hill_trait_3,hill_trait_4,hill_phy_1,hill_phy_2,hill_phy_3,hill_phy_4,richness,iteration
1,1.000000,1.000000,1.000000,1.000000,1.0,inf,inf,inf,NaN,NaN,NaN,NaN,1,2,3,4,1,0.0
2,1.000000,1.000000,1.000000,1.000000,1.0,inf,inf,inf,NaN,NaN,NaN,NaN,1,2,3,4,1,10.0
3,1.102592,1.034626,1.025973,1.023054,1.0,inf,inf,inf,11.032254,10.971902,10.964029,10.961361,1,2,3,4,3,20.0
4,1.181878,1.070583,1.053081,1.047055,1.0,inf,inf,inf,6.792398,6.571370,6.502872,6.468026,1,2,3,4,3,30.0
5,1.262846,1.108246,1.081396,1.072058,1.0,inf,inf,inf,7.499219,7.029423,6.778752,6.573916,1,2,3,4,3,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,17.521505,12.704380,10.642635,9.609134,1.0,inf,inf,inf,18.433256,13.473036,11.256117,10.098081,1,2,3,4,28,1960.0
198,17.210243,12.257042,10.231898,9.254691,1.0,inf,inf,inf,18.044482,12.934295,10.744881,9.640783,1,2,3,4,28,1970.0
199,16.753050,11.760135,9.814867,8.895905,1.0,inf,inf,inf,17.385387,12.358706,10.192044,9.151311,1,2,3,4,28,1980.0
200,16.441066,11.760135,9.766883,8.775459,1.0,inf,inf,inf,17.116980,12.465008,10.253494,9.131814,1,2,3,4,26,1990.0


In [49]:
columns = ["richness", "hill_abund_1", "hill_abund_2", "hill_abund_3", "hill_abund_4", "hill_trait_1", "hill_trait_2", "hill_trait_3", "hill_trait_4"]
dataset = simulator.generate_dataset(prior=prior, samples=40, columns=columns)

theta = dataset[0]
x = dataset[1]
print("theta.size", theta.shape)
print("x.size", x.shape)

theta.size torch.Size([7986, 2])
x.size torch.Size([7986, 9])


In [50]:
from sbi.inference import SNPE

snpe = SNPE(prior=prior.get_joint_uniform())
density_estimator = snpe.append_simulations(theta, x).train()
posterior = snpe.build_posterior(density_estimator)

 Neural network successfully converged after 300 epochs.

In [61]:
theta_true, x_obs = simulator.generate_dataset(prior=prior, samples=1, columns=columns)
theta_true, x_obs = theta_true[0], x_obs[-1]
print("Observed simulation output:", x_obs)

# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = posterior.sample((1000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)

Observed simulation output: tensor([19.5154, 12.7301,  9.6081,  8.0555, 23.3418, 16.1277, 13.1758, 11.7195,
        33.0000])


Drawing 1000 posterior samples: 1091it [00:00, 201958.94it/s]           

Posterior samples shape: torch.Size([1000, 2])
Posterior mean estimate: tensor([151.3894, 678.5510])

Theta True      : tensor([127.0089, 765.5049])
Theta Prediction: tensor([151.3894, 678.5510])


In [ ]:

# prior = {
#     "individuals_local": int,
#     "individuals_meta": int,
#     "species_meta": int,
#     "speciation_local": float,
#     "speciation_meta": float,
#     "extinction_meta": float,
#     "env_sigma": float,
#     "trait_sigma": float,
#     "comp_sigma": float,
#     "dispersal_prob": float,
#     "mutation_rate": float,
#     "equilib_escape": int,
#     "num_basepairs": int,
#     "init_type": str,
#     "niter": int,
#     "niterTimestep": int,
# }

# torch
# human readable discrete

# generate dataset

# class ChoiceDistribution(Distribution):
#     def __init__(self, choices: list):
#         super().__init__(0, len(choices))  # Initialize base class
#         self.choices = choices



In [ ]:
import torch
from torch.distributions import Uniform



# Lower bounds for the 15 numeric parameters (order unchanged):
prior_low = torch.tensor([
    50.0,        # individuals_local (J): local community individuals
    1000.0,      # individuals_meta (JM): metacommunity individuals
    20.0,        # species_meta (SM): metacommunity species richness
    0.0,         # speciation_local (ν): local speciation rate in [0,1]
    0.001,       # speciation_meta (Λ): metacommunity speciation rate
    0.0,         # extinction_meta (Ε): extinction rate in [0,1]
    0.01,        # env_sigma: metacommunity trait evolution variance (σ²)
    0.01,        # trait_sigma: local trait evolution variance (σ²)
    0.1,         # comp_sigma: strength of ecological filtering (SE)
    0.0,         # dispersal_prob (m): immigration rate from metacommunity in [0,1]
    1e-9,        # mutation_rate (µ): mutation rate
    1.0,         # equilib_escape (α): abundance/Ne scaling factor
    100.0,       # num_basepairs (L): sequence length (bp)
])

# Upper bounds for the 15 numeric parameters:
prior_high = torch.tensor([
    1000.0,      # individuals_local (J)
    10000.0,     # individuals_meta (JM)
    80.0,        # species_meta (SM)
    1.0,         # speciation_local (ν)
    1.0,         # speciation_meta (Λ)
    1.0,         # extinction_meta (Ε)
    5.0,         # env_sigma (σ² metacommunity)
    5.0,         # trait_sigma (σ² local)
    10.0,        # comp_sigma (SE)
    1.0,         # dispersal_prob (m)
    1e-6,        # mutation_rate (µ)
    100.0,       # equilib_escape (α)
    10000.0,     # num_basepairs (L)
])

# Create the joint uniform prior over these 15 parameters.
joint_prior = Uniform(low=prior_low, high=prior_high)

# Example: Draw 5 samples from this joint prior.
samples = joint_prior.sample()
print(samples)
